In [1]:
############################# this code unfinished #######################
# seperately apply marginal interal sampling on different households
import tensorflow as tf
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
import matplotlib as mp
import random as rd
import argparse
import os, sys
import csv
import math
import time
import matplotlib.pyplot as pl
from numpy import random, histogram2d, diff
from scipy.interpolate import interp2d
#%matplotlib inline
# class County:
#     def __init__(self,parsename):
#         self.parsename = parsename
#         dataframe = xls.parse(parsename)
#         self.data = dataframe
#     def disp_all(self):
#         print self.dataframe
#     def get_all(self):
#         return self.dataframe
# xls = pd.ExcelFile('No-drybulb-dewpoint-short-dataset-CT.xlsx',header = None)
# Zonal = []
# Zonal.append(County('CT'))
# ZonalNum = 1

print "setting parameters ............."
num_epochs = 1000 # training epoches for each customer samples
num_realisations = 5000
out_thresh = num_epochs - 100
day_steps_f = 48
#val_rate_f = 0.15
test_batch_size_f = 28*day_steps_f # days of a batch
valid_batch_size_f = 28*day_steps_f
train_batch_size_f = 28*day_steps_f
n_output_f = 1
n_hidden_f_1 = 100
n_hidden_f_2 = 100
n_hidden_f_3 = 100
n_hidden_f_4 = 100
tao_f = 0.1
gap_test_f = 10
batch_size_f = test_batch_size_f # in this version, batch_size set same
preserve_f = 0#16114 ## amount of first time points without complete features
_dropout_train = 0.5
_dropout_test = 1.0
ZonalNum = 1
# DEMAND MATRIX 9 X LENGTH, 9: INC is total, index with 0, other substations are from 1 -> 8



############################||||||||||||||||||||||||||data loading

print "loading data ..........."
#ISO_name = 'No-drybulb-dewpoint-short-dataset-CT.csv'
#HOME_name = 'data/MAC005540.csv'
count = 0
hid = 0
for root, dirs, filenames in os.walk('./data/'):
    for fname in filenames:
        dbslice = pd.read_csv('./data/' + fname)
        if count < hid:
            count = count +1
            continue;
        if count == hid:
            xls = dbslice
        else:
            xls = pd.concat([xls,dbslice], axis = 0)
        count = count + 1
        if count > hid:
            print 'reach max file number'
            break;
xls.shape
rows_f = xls.shape[0]
columns_f = xls.shape[1]
database_f = np.array(xls)
np.random.shuffle(database_f)
for i in range(rows_f):
    for j in range(columns_f):
        database_f[i,j] = np.float(database_f[i,j])
totalen_f = rows_f
#print database_f[:,0]
n_input_f = columns_f - 1
data_norm = np.max(database_f, axis = 0)
database_f = database_f/data_norm
#print totalen_f
db_f = database_f
#print db_f
#define id arrays
test_id_f = np.array(test_batch_size_f)
valid_id_f = np.array(2*valid_batch_size_f)
train_id_f = np.array(totalen_f - test_batch_size_f - valid_batch_size_f)

#give values to id arrays
rang = range(preserve_f, totalen_f - test_batch_size_f)
valid_id_f = rd.sample(rang,2*valid_batch_size_f)
test_id_f = np.array(range(totalen_f - test_batch_size_f,totalen_f))
train_id_f = set(range(preserve_f, totalen_f - test_batch_size_f)) - set(valid_id_f)

#sort three id array
valid_id_f = np.sort(valid_id_f)
test_id_f = np.sort(test_id_f)
train_id_f = np.array(list(train_id_f))



setting parameters .............
loading data ...........
reach max file number


In [2]:
testset = xls[-48*28:]
trainset = xls[0:-48*28]

In [3]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.regression.quantile_regression import QuantReg
mod = smf.quantreg('Power ~ monday + tuesday + wednesday + thursday + \
friday + saturday + holiday+ temperature + humidity + windspeed + rainfall+\
h1 + h2 + h3 + h4 + h5 + h6 + h9 + h10 + h11 + h12 + h13 + h14 + h15 + h16 \
+ h17 + h18 + h19 + h20 + h21 + h22 + h23 + h24+ july + aug + sep + jan + feb + mar\
+apr + may + june ', trainset)
def pinball_loss(A,B,tao):
	cost = 0.0
	A = A.reshape([-1])
	B = B.reshape([-1])
	for i in range(A.shape[0]):
	    if A[i]-B[i]>=0:
	        tmp = (A[i]-B[i])*(tao)
	    else:
	        tmp = (B[i]-A[i])*(1.0-tao)
	    cost = tmp+cost
	return cost
quantiles = [.1, .2, .3, .4, .5, .6, .7, .8, .9]

# get all result instances in a list
res_all = [mod.fit(q=q) for q in quantiles]
Plist = []
ind = 0
totalpb = 0
for qm, res in zip(quantiles, res_all):
    # get prediction for the model and plot
	qforecasts = res.predict({'jan': testset['jan'], 'feb': testset['feb'], 'mar': testset['mar'], \
    'apr': testset['apr'], 'may': testset['may'], 'june': testset['june'], \
    'july': testset['july'], 'aug': testset['aug'], 'sep': testset['sep'], \
    'temperature': testset['temperature'], 'humidity': testset['humidity'], 'windspeed': testset['windspeed'], \
    'rainfall': testset['rainfall'],'monday': testset['monday'], 'tuesday': testset['tuesday'], \
    'h1': testset['h1'], 'h2': testset['h2'], 'h3': testset['h3'], \
    'h4': testset['h4'], 'h5': testset['h5'], 'h6': testset['h6'], \
    'h7': testset['h7'], 'h8': testset['h8'], 'h9': testset['h9'], \
    'h10': testset['h10'], 'h11': testset['h11'], 'h12': testset['h12'], \
    'h13': testset['h13'], 'h14': testset['h14'], 'h15': testset['h15'], \
    'h16': testset['h16'], 'h17': testset['h17'], 'h18': testset['h18'], \
    'h19': testset['h19'], 'h20': testset['h20'], 'h21': testset['h21'], \
    'h22': testset['h22'], 'h23': testset['h23'], 'h24': testset['h24'], \
    'wednesday': testset['wednesday'], 'thursday': testset['thursday'], 'friday': testset['friday']\
    , 'saturday': testset['saturday'], 'holiday': testset['holiday']})
	Plist.append(qforecasts)
	loss_pb = pinball_loss(testset['Power'], qforecasts, qm)
	print '%.4f'%(loss_pb)
	totalpb = totalpb + loss_pb
print totalpb/9

7.5641
14.8784
21.2331
26.5136
30.9246
34.3252
36.8348
37.0776
31.1656
26.7241081096


In [23]:
print 'Visualization'
N = 1
x = np.linspace(0, 336, 336)
oneday_x = x
vals = [1,2,3,4] # Values to iterate over and add/subtract from y.
pl.rc('font', family='serif')
fig = pl.figure(figsize=(8, 6))
ax = fig.add_subplot(1, 1, 1)
ax.set_xlabel('The x values')
ax.set_ylabel('The y values')
#ax.set_ylim([0,1.8])
pl.rc('font', family = 'serif', serif = 'Times')
pl.rc('xtick', labelsize = 8)
pl.rc('ytick', labelsize = 8)
pl.rc('axes', labelsize = 8)
#################################### one day case
#for i, val in enumerate(vals):
#    alpha = 0.5*(i+1)/len(vals) # Modify the alpha value for each iteration.
#    if i == 0:
#        ax.fill_between(oneday_x, Plist[8][0:336], Plist[0][0:336], color='red', alpha=alpha*0.7)
#    elif i == 1:
#        ax.fill_between(oneday_x, Plist[7][0:336], Plist[1][0:336], color='red', alpha=alpha*0.7)
#    elif i == 2:
#        ax.fill_between(oneday_x, Plist[6][0:336], Plist[2][0:336], color='red', alpha=alpha*0.7)
#    else:
#        ax.fill_between(oneday_x, Plist[5][0:336], Plist[3][0:336], color='red', alpha=alpha*0.7)
ax.plot(oneday_x/48+0.5, testset['Power'][0:336], '-', color='black',linewidth=1.) # Plot the original signal
boxP = np.array(Plist)
boxP = boxP.T
boxP = boxP.reshape([7,-1])
boxP = boxP.T
pd.DataFrame(boxP).boxplot()
pl.show()


Visualization


/home/codefisheng/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:32: FutureWarning: 
The default value for 'return_type' will change to 'axes' in a future release.
 To use the future behavior now, set return_type='axes'.
 To keep the previous behavior and silence this warning, set return_type='dict'.


In [5]:
boxP = np.array(Plist)
boxP = boxP.T
boxP = boxP.reshape([7,-1])
boxP = boxP.T
pd.DataFrame(boxP).boxplot()
ax.plot(oneday_x/48, testset['Power'][0:336], color='black') # Plot the original signal
pl.show()

/home/codefisheng/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:5: FutureWarning: 
The default value for 'return_type' will change to 'axes' in a future release.
 To use the future behavior now, set return_type='axes'.
 To keep the previous behavior and silence this warning, set return_type='dict'.


In [6]:
help(pl.boxplot())

336